<a href="https://colab.research.google.com/github/saranyapichandi96/Task2/blob/main/studentmongo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pymongo
import json
from bson.objectid import ObjectId
#First create a database and then load the student.json dataset.
mydb=myclient['studentdb']

mycollection=mydb['studentsdetails']

with open('students.json') as f:
    data = [json.loads(line) for line in f]

#Insert the students record into the collection.
id_doc=mycollection.insert_many(data)
for data in mycollection.find(): 
    print(data)

#1)Find the student name who scored maximum scores in all (exam, quiz and homework)?
data=mycollection.aggregate([{ "$unwind": "$scores" },
                             { "$group": {"_id": "$scores.type",
                               "nameScores": {"$push": { "name": "$name", "score": "$scores.score" }},
                               "max": { "$max": "$scores.score" }}},
                             { "$set": {"max_score": {"$first": {"$filter": {"input": "$nameScores",
                                                                             "cond": { "$eq": [ "$$this.score", "$max" ] }}}}}},
                             { "$project": { "_id": 0,
                                            "type": "$_id",
                                            "max_score": 1}}])
print(list(data))

#2)Find students who scored below average in the exam and pass mark is 40%?
data=mycollection.aggregate([
    {"$unwind":"$scores"},
    {"$match":{"scores.type":"exam"}},
    {"$group": {
        "_id": "null",
        "failCount_exam": {"$sum": { "$cond": [{ "$lt": ["$scores.score", 40 ] }, 1, 0 ],
        }}}},
    {"$project": {
        "_id" : 0,
       
        "failCount_exam" : 1,
        }}
    ])
print(list(data)
#3)Find students who scored below pass mark and assigned them as fail, and above pass mark as pass in all the categories.fail/pass
data_e=mycollection.aggregate([{"$project": {"results": {"$arrayElemAt": [{"$filter": 
                                                                        {"input": "$scores",
                                                                         "as": "res",
                                                                         "cond": {"$eq": ["$$res.type", "exam"],
                                                                                  }}},0]}}},
                             {
                                 "$project": {
                                "exam_fail": { "$gt": ["$exam_result.score",40] }}
                                }])
print("failmarks in exam :\n",list(data))

data_q=mycollection.aggregate([{"$project": {"results": {"$arrayElemAt": [{"$filter": 
                                                                        {"input": "$scores",
                                                                         "as": "res",
                                                                         "cond": {"$eq": ["$$res.type", "quiz"],
                                                                                  }}},0]}}},
                             {
                                 "$project": {
                                "quiz_fail": { "$gt": ["$quiz_result.score",40] }}
                                }])
print("failmarks in quiz :\n",list(data))

data_h=mycollection.aggregate([{"$project": {"results": {"$arrayElemAt": [{"$filter": 
                                                                        {"input": "$scores",
                                                                         "as": "res",
                                                                         "cond": {"$eq": ["$$res.type", "quiz"],
                                                                                  }}},0]}}},
                             {
                                 "$project": {
                                "homework_fail": { "$gt": ["$homework_result.score",40] }}
                                }])
print("failmarks in homework :\n",list(data))
#4)Find the total and average of the exam, quiz and homework and store them in a separate collection.
#Find the total 
total=mycollection.aggregate([
   {
     "$project": {
       "Total": { "$sum": "$scores.score"},
       
     }
   }
])
print(list(total))
#store them in a separate collection.
total_id=mycollection.insert_one(total)

#Find the avg
avarage=mycollection.aggregate([
   { "$project": { 
       "Avg": { "$avg": "$scores.score"} } } 
])
print(list(avarage))
#store them in a separate collection.
#creating a new_collection which contains avg and total
new_collection=mydb['avg_details']
new_collection.insert_many(avg)
#i'm droping the id becoz both conntaing same values
dat=new_collection.update({}, {'$unset': {'tags._id':1}}, multi=True)
new_collection.insert_many(total)
#5)Create a new collection which consists of students who scored below average and above 40% in all the categories.
mycollection.find({"scores.score" : { '$match':{"scores.score":{'$gt' :40 , '$lt' :'100'}}}})

#6)Create a new collection which consists of students who scored below the pass mark in all the categories.
#fail marks in exam
#creating a collection
collection=mydb['passmark']
d_e=mycollection.find("exam_fail"=="False")
collection.insert_many(d_e)
for i in d:
    print(i)
dr=new_collection.update({}, {'$unset': {'tags._id':1}}, multi=True)
collection.insert_many(d_e)
d_q=mycollection.find("quiz_fail"=="False")
collection.insert_many(d_q)
d_h=mycollection.find("homework_fail"=="False")
collection.insert_many(d_h)
#7)Create a new collection which consists of students who scored below the fail mark in all the categories.
#fail marks in exam
#creating a collection
collections=mydb['failmark']
d_fe=mycollection.find("exam_fail"=="True")
collection.insert_many(d_e)
for i in d:
    print(i)
dr=new_collection.update({}, {'$unset': {'tags._id':1}}, multi=True)
collection.insert_many(d_fe)
d_fq=mycollection.find("quiz_fail"=="True")
collection.insert_many(d_fq)
d_fh=mycollection.find("homework_fail"=="True")
collection.insert_many(d_fh)


